<a href="https://colab.research.google.com/github/Mustafa1x/Assignment-2/blob/master/Copy_of_Project1_starter_NCAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mustafa Ibrahim 
#Project 1

# Project \#1: NCAA March Madness Modeling

## The Data:
- Ken Pomeroy (https://kenpom.com/), 
  - 'kenpom_2021.csv' has a subset of the data from kenpom.com
- FiveThirtyEight: https://projects.fivethirtyeight.com/2021-march-madness-predictions/  
  - '538ratingsMen.csv' has FiveThirtyEight's rankings
- Massey Ratings: https://www.masseyratings.com/cb/ncaa-d1/ratings 
- Kaggle Machine Learning Mania 2021: https://www.kaggle.com/c/ncaam-march-mania-2021
- You can use external data as well, the Kaggle competition has several other linked datasets you can use. 

Helpful Articles:
- https://fivethirtyeight.com/methodology/how-our-march-madness-predictions-work-2/ 

In [14]:
from google.colab import drive
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn import neighbors, tree, metrics
drive.mount('/content/drive')
kenpom = pd.read_csv('/content/drive/MyDrive/Dataset/kenpom_2021.csv')
data_538 = pd.read_csv('/content/drive/MyDrive/Dataset/538ratingsMen.csv')
train_test_data = pd.read_csv('/content/drive/MyDrive/Dataset/fivethirtyeight_ncaa_forecasts.csv')

pd.set_option('display.max_columns', 100)

kenpom.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Season,TeamID,adj_o,adj_d,luck,rank,adj_em
0,2011,1326,125.4,88.4,0.043,1,0.9824
1,2011,1181,118.8,87.2,0.006,2,0.9720
2,2011,1242,119.8,88.3,0.071,3,0.9711
3,2011,1400,114.0,85.3,-0.055,4,0.9657
4,2011,1345,116.1,87.2,-0.004,5,0.9641


In [5]:
pd.set_option('display.max_columns', 100)

data_538.head()

,Season,TeamID,TeamName,538rating
0,2016,1242,Kansas,94.46
1,2016,1314,North Carolina,93.94
2,2016,1438,Virginia,92.46
3,2016,1277,Michigan State,91.84
4,2016,1328,Oklahoma,89.96


## Here I am checking if the kenpom data has any null values in the specified columns.

In [6]:
print(kenpom['Season'].isna().any())

print(kenpom['TeamID'].isna().any())

print(kenpom['adj_o'].isna().any())

print(kenpom['adj_d'].isna().any())

print(kenpom['luck'].isna().any())

print(kenpom['rank'].isna().any())

print(kenpom['adj_em'].isna().any())

False
False
False
False
False
False
False


## Here I am checking if the data_538 data has any null values in the specified columns.

In [7]:
print(data_538['Season'].isna().any())

print(data_538['TeamID'].isna().any())

print(data_538['TeamName'].isna().any())

print(data_538['538rating'].isna().any())



False
False
False
False


## Here I created a KNN function that can give you the teams expected rating depending on the input. Here I demonstrated it works with the input 2.

In [12]:
def knn(new_rating, data, k):

  data_copy= data.copy()

  data_copy['distance_to_new'] = numpy.sqrt((new_rating['Season']-data['Season']) ** 2 +(new_rating['TeamID']-data['TeamID']) ** 2 + (new_rating['538rating']-data['538rating']) ** 2) 
  
  sorted_data = data_copy.sort_values(['distance_to_new']) 
  
  return sorted_data.iloc[0:k]['538rating'].mean()

In [13]:
new_rating = {}

new_rating['Season'] = 2020

new_rating['TeamID'] = 1242

new_rating['538rating'] = 94.46

print(knn(new_rating, data_538, 2))

93.32


## Decision tree:

In [32]:
predictors = ['team_rating','team_id','rd1_win', 'rd2_win', 'rd3_win', 'rd4_win',
              'rd5_win', 'rd6_win', 'rd7_win', 'team_alive'] 
target = "team_seed"

train_data, test_data, train_sln, test_sln =  train_test_split(train_test_data[predictors], train_test_data[target], test_size = 0.2, random_state=41) 

In [33]:
dt = tree.DecisionTreeRegressor()

dt.fit(train_data, train_sln)

dt_predictions = dt.predict(test_data)

print(metrics.r2_score(test_sln, dt_predictions)) 

ValueError: ignored